# Test mutual information estimators

## Preamble

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
font = {'family' : 'DejaVu Sans',
        'size'   : 18}

matplotlib.rc('font', **font)

In [ ]:
import os
import json
import csv

from datetime import datetime

In [ ]:
from pathlib import Path
path = os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), "../../data/"))

In [ ]:
experiments_path = path + "/mutual_information/synthetic/"

#### Importing the module

In [ ]:
import mutinfo.estimators.mutual_information as mi_estimators
from mutinfo.utils.dependent_norm import multivariate_normal_from_MI

In [ ]:
### SETTINGS ###
%run ./Settings.ipynb

#### Standard tests with arbitrary mapping

In [ ]:
def perform_normal_compressed_test(mi, n_samples, X_dimension, Y_dimension, X_map=None, Y_map=None,
                                   X_compressor=None, Y_compressor=None, verbose=0):
    # Generation.
    random_variable = multivariate_normal_from_MI(X_dimension, Y_dimension, mi)
    X_Y = random_variable.rvs(n_samples)
    X = X_Y[:, 0:X_dimension]
    Y = X_Y[:, X_dimension:X_dimension + Y_dimension]
        
    # Mapping application.
    if not X_map is None:
        X = X_map(X)
           
    if not Y_map is None:
        Y = Y_map(Y)
        
    # Mutual information estimation.
    mi_estimator = mi_estimators.MutualInfoEstimator(entropy_estimator_params=entropy_estimator_params)
    mi_estimator.fit(X, Y, verbose=verbose)
    mi = mi_estimator.estimate(X, Y, verbose=verbose)
    
    # Mutual information estimation for compressed representation.
    mi_estimator = mi_estimators.LossyMutualInfoEstimator(X_compressor, Y_compressor,
                                                          entropy_estimator_params=entropy_estimator_params)
    mi_estimator.fit(X, Y, verbose=verbose)
    mi_compressed = mi_estimator.estimate(X, Y, verbose=verbose)
    
    return mi, mi_compressed

In [ ]:
def perform_normal_compressed_tests_MI(MI, n_samples, X_dimension, Y_dimension, X_map=None, Y_map=None,
                                       X_compressor=None, Y_compressor=None, verbose=0):
    """
    Estimate mutual information for different true values
    (transformed normal distribution).
    """
    n_exps = len(MI)
    
    # Mutual information estimates.
    estimated_MI = []
    estimated_MI_compressed = []

    # Conducting the tests.
    for n_exp in range(n_exps):
        print("\nn_exp = %d/%d\n------------\n" % (n_exp + 1, n_exps))
        mi, compressed_mi = perform_normal_compressed_test(MI[n_exp], n_samples, X_dimension, Y_dimension,
                                                           X_map, Y_map, X_compressor, Y_compressor, verbose)
        estimated_MI.append(mi)
        estimated_MI_compressed.append(compressed_mi)
        
    return estimated_MI, estimated_MI_compressed

In [ ]:
def plot_estimated_compressed_MI(MI, estimated_MI, estimated_MI_compressed, title):
    estimated_MI_mean = np.array([estimated_MI[index][0] for index in range(len(estimated_MI))])
    estimated_MI_std  = np.array([estimated_MI[index][1] for index in range(len(estimated_MI))])
    
    estimated_MI_compressed_mean = np.array([estimated_MI_compressed[index][0]
                                             for index in range(len(estimated_MI_compressed))])
    estimated_MI_compressed_std  = np.array([estimated_MI_compressed[index][1]
                                             for index in range(len(estimated_MI_compressed))])
    
    fig_normal, ax_normal = plt.subplots()

    fig_normal.set_figheight(11)
    fig_normal.set_figwidth(16)

    # Grid.
    ax_normal.grid(color='#000000', alpha=0.15, linestyle='-', linewidth=1, which='major')
    ax_normal.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')

    ax_normal.set_title(title)
    ax_normal.set_xlabel("$I(X,Y)$")
    ax_normal.set_ylabel("$\\hat I(X,Y)$")
    
    ax_normal.minorticks_on()
    
    #ax_normal.set_yscale('log')
    #ax_normal.set_xscale('log')

    ax_normal.plot(MI, MI, label="$I(X,Y)$", color='red')
    
    ax_normal.plot(MI, estimated_MI_mean, label="$\\hat I(X,Y)$")
    ax_normal.fill_between(MI, estimated_MI_mean + estimated_MI_std, estimated_MI_mean - estimated_MI_std, alpha=0.2)
    
    ax_normal.plot(MI, estimated_MI_compressed_mean, label="$\\hat I_{compr}(X,Y)$")
    ax_normal.fill_between(MI, estimated_MI_compressed_mean + estimated_MI_compressed_std,
                           estimated_MI_compressed_mean - estimated_MI_compressed_std, alpha=0.2)

    ax_normal.legend(loc='upper left')

    ax_normal.set_xlim((0.0, None))
    ax_normal.set_ylim((0.0, None))

    plt.show();

### Global parameters

In [ ]:
# The values of mutual information under study.
MI = np.linspace(0.0, 10.0, 41)
n_exps = len(MI)

# Sample size and dimensions of vectors X and Y.
n_samples = 5000

### Images of rectangles

In [ ]:
from mutinfo.utils.synthetic import normal_to_rectangle_coords
from mutinfo.utils.synthetic import rectangle_coords_to_rectangles

In [ ]:
X_dimension = 4
Y_dimension = 4
latent_dimension = 4

min_delta = 2
img_width = 32
img_height = 32

experiments_dir = ('rectangles_%dx%d' % (img_width, img_height))

In [ ]:
def imshow_array(array):
    """Display array of pixels."""
    plt.axis('off')
    plt.imshow((255.0 * array).astype(np.uint8), cmap=plt.get_cmap("gray"), vmin=0, vmax=255)

#### Train the autoencoder

In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
n_train_samples = 6000
n_test_samples  = 1000

In [ ]:
random_variable = multivariate_normal()
X = random_variable.rvs((n_train_samples + n_test_samples, X_dimension))
X = rectangle_coords_to_rectangles(normal_to_rectangle_coords(X, min_delta, img_width, min_delta, img_height), img_width, img_height)
X_train = X[0:n_train_samples]
X_test  = X[n_train_samples:n_train_samples + n_test_samples]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=latent_dimension).fit(X_train.reshape(X_train.shape[0], -1))

In [ ]:
def rectangles_mapping(X):
    """ Map Gaussian vector to the coordinates of the rectangle. """
    return normal_to_rectangle_coords(X, min_delta, img_width, min_delta, img_height)

def rectangles_compressor(X):
    """ Parameters to images, then to latent representations. """
    images = rectangle_coords_to_rectangles(X, img_width, img_height)
    return pca.transform(images.reshape(images.shape[0], -1))

In [ ]:
estimated_MI, estimated_MI_compressed = perform_normal_compressed_tests_MI(MI,
    n_samples, X_dimension, Y_dimension, rectangles_mapping, rectangles_mapping,
    rectangles_compressor, rectangles_compressor, verbose=10)

In [ ]:
plot_estimated_compressed_MI(MI, estimated_MI, estimated_MI_compressed, "Rectangles")

In [ ]:
save_estimated_MI(MI, estimated_MI, experiments_dir + '/coordinates')
save_estimated_MI(MI, estimated_MI_compressed, experiments_dir + '/compressed/PCA')

In [ ]:
print("OK")